In [1]:
%load_ext autoreload
%autoreload 2

import formatOrtho
import pyhamPipeline

import pyham
from pyoma.browser import db 
import numpy as np
from tables import *
import re
from ete3 import Tree
import pickle
import tempfile
import functools
import config

h5file = open_file(config.omadirLaurent + 'OmaServer.h5', mode="r")
dbObj = db.Database(h5file)
omaIdObj = db.OmaIdMapper(dbObj)

/home/laurent/anaconda3/lib/python3.6/site-packages/tables/leaf.py:396: PerformanceWarning: The Leaf ``/Protein/_i_Entries/OmaHOG/sorted`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)


In [138]:
species_tree, replacement_dico = formatOrtho.fix_species_tree("speciestree.nwk", omaIdObj)

In [139]:
ortho = dbObj.get_orthoxml(100000).decode()

In [140]:
newortho = formatOrtho.convert_orthoxml_ids(ortho, replacement_dico)

In [141]:
hamObj = pyham.Ham(species_tree, newortho, type_hog_file="string", use_internal_name = True)

In [60]:
print(hamObj.get_dict_top_level_hogs())


{'100000': <HOG(100000)>}


In [107]:
hog21 = hamObj.get_hog_by_id('100000')
tp = hamObj.create_tree_profile(hog=hog21)

In [62]:
tp.treemap

Tree node 'rosids' (-0x7ffff80dd97ec1c4)

In [63]:

for node in tp.treemap.traverse():
    if not node.is_root():
        print(node.name)
        print('N')
        print(node.nbr_genes)
        print('D')
        print(node.dupl)
        print('L')
        print(node.lost)
        print('I')
        print(node.identical)


fabids
N
1
D
0
L
0
I
1
malvids
N
1
D
0
L
0
I
1
Vitis vinifera
N
0
D
0
L
1
I
0
Papilionoideae
N
1
D
0
L
0
I
1
Malpighiales
N
0
D
0
L
1
I
0
Prunus persica
N
0
D
0
L
1
I
0
Brassicaceae
N
1
D
0
L
0
I
1
Malvaceae
N
0
D
0
L
1
I
0
Lotus japonicus
N
0
D
0
L
1
I
0
Glycine max
N
0
D
0
L
1
I
0
Medicago truncatula
N
1
D
0
L
0
I
1
Manihot esculenta
N
0
D
0
L
0
I
0
Populus trichocarpa
N
0
D
0
L
0
I
0
Brassica
N
1
D
0
L
0
I
1
Arabidopsis
N
0
D
0
L
1
I
0
Theobroma cacao
N
0
D
0
L
0
I
0
Gossypium hirsutum
N
0
D
0
L
0
I
0
Brassica napus
N
1
D
0
L
0
I
1
Brassica oleracea
N
1
D
0
L
0
I
1
Arabidopsis lyrata
N
0
D
0
L
0
I
0
Arabidopsis thaliana
N
0
D
0
L
0
I
0


In [35]:
T = Tree(species_tree , format =1 )


In [36]:
mapping = {}
mappingINV = {}
for i,node in enumerate(T.traverse()):
    mapping[i] = node.name
    mappingINV[node.name] = i


In [37]:
import datasketch
import ete3
import sparse
import itertools

In [130]:

def Tree2Hashes(treemap):
    #turn each tree into a minhash object
    #serialize and store as array
    eventdict = { 'presence':[] , 'gain':[] , 'loss':[] , 'duplication':[]}	

    for node in treemap.traverse():

        if not node.is_root():
            print(node.name)
            if node.nbr_genes >0:
                eventdict['presence'].append('P'+node.name)
            if node.dupl > 0:
                eventdict['duplication'].append('D'+node.name)
            if node.lost > 0:
                eventdict['loss'].append('L'+node.name)
        else:
            eventdict['gain'].append('G'+node.name)

    hashes = []

    hashesDict = {}

    for array in eventdict:
        eventdict[array] = set(eventdict[array])

        minHash = datasketch.MinHash(num_perm=128)

        for element in eventdict[array]:

            minHash.update(element.encode())

        hashesDict[array] = minHash

        lminHash = datasketch.LeanMinHash(minHash)

        buf = bytearray(lminHash.bytesize())
        lminHash.serialize(buf)
        hashes.append([buf])

    for j in range(1,len(eventdict.keys())):
        for i in itertools.combinations(eventdict.keys(), j+1):
            combName = ''
            minHash = datasketch.MinHash(num_perm=128)
            for array in i:
                combName += array
                minHash.merge(hashesDict[array])

            hashesDict[combName] = minHash

            lminHash = datasketch.LeanMinHash(minHash)

            buf = bytearray(lminHash.bytesize())
            lminHash.serialize(buf)
            hashes.append([buf])

    hashmat = np.vstack(hashes)
    return hashmat, hashesDict


hashmat, hashesDict = Tree2Hashes(tp.treemap)
print(hashmat.shape)
print(hashmat)

fabids
malvids
Vitis vinifera
Papilionoideae
Malpighiales
Prunus persica
Brassicaceae
Malvaceae
Lotus japonicus
Glycine max
Medicago truncatula
Manihot esculenta
Populus trichocarpa
Brassica
Arabidopsis
Theobroma cacao
Gossypium hirsutum
Brassica napus
Brassica oleracea
Arabidopsis lyrata
Arabidopsis thaliana
(15, 524)
[[  1   0   0 ... 174 193  24]
 [  1   0   0 ...  79  24 238]
 [  1   0   0 ...  53  76 106]
 ...
 [  1   0   0 ... 174 193  24]
 [  1   0   0 ...  53  76 106]
 [  1   0   0 ... 174 193  24]]


In [131]:
hashesDict

{'duplication': <datasketch.minhash.MinHash at 0x7f228c9db0f0>,
 'gain': <datasketch.minhash.MinHash at 0x7f228c9dbfd0>,
 'gainduplication': <datasketch.minhash.MinHash at 0x7f228c9db160>,
 'gainloss': <datasketch.minhash.MinHash at 0x7f228c9db4e0>,
 'gainlossduplication': <datasketch.minhash.MinHash at 0x7f228c9db710>,
 'loss': <datasketch.minhash.MinHash at 0x7f228c9db2b0>,
 'lossduplication': <datasketch.minhash.MinHash at 0x7f228c9dbba8>,
 'presence': <datasketch.minhash.MinHash at 0x7f228c9dbda0>,
 'presenceduplication': <datasketch.minhash.MinHash at 0x7f228c9dbe80>,
 'presencegain': <datasketch.minhash.MinHash at 0x7f228c9dbc50>,
 'presencegainduplication': <datasketch.minhash.MinHash at 0x7f228c9dba90>,
 'presencegainloss': <datasketch.minhash.MinHash at 0x7f228c9dbac8>,
 'presencegainlossduplication': <datasketch.minhash.MinHash at 0x7f228c9db748>,
 'presenceloss': <datasketch.minhash.MinHash at 0x7f228c9dbb70>,
 'presencelossduplication': <datasketch.minhash.MinHash at 0x7f22

In [99]:
import numpy as np
from scipy.sparse import csr_matrix

def Tree2mat(eteobj,taxaIndex):
    #use partials to configure the taxa index
    #turn each tree into a sparse matrix with 4 rows
    rowdict={ 'presence':0 , 'gain':1 , 'loss':2 , 'duplication':3}
    
    mat = csr_matrix( (len(rowdict), len(taxaIndex) ) )
    
    coords = []
    for node in eteobj.traverse():
    # traverse() returns an iterator to traverse the tree structure
    # strategy:"levelorder" by default; nodes are visited in order from root to leaves
    # it return treeNode instances
        if not node.is_root():            
            if node.nbr_genes >0:
                mat[ rowdict['presence'] , taxaIndex[node.name]] =1 
            if node.lost > 0:
                mat[ rowdict['loss'] , taxaIndex[node.name]] =1
            if node.dupl > 0:
                mat[ rowdict['duplication'] , taxaIndex[node.name]] =1
        else:
            mat[ rowdict['gain'] , taxaIndex[node.name]] =1
  
    return mat

print(np.sum(Tree2mat(tp.treemap, mappingINV)))



16.0


/home/laurent/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [14]:
import pyham
from pyoma.browser import db 
import numpy as np
from tables import *
import re
from ete3 import Tree
import pickle
import tempfile
import functools
import config

import pyhamPipeline
import profileGen

# with open( config.working_dir + "speciestree.nwk" , 'r') as treefile:
#     species_tree = treefile.read()
# t = Tree(species_tree, format =1 )
# print(t)
# #rewrite IDs and reload tree
# #
# species_tree = pyham.utils.get_newick_string(working_dir + "speciestree_hack.nwk", type="nwk")


#open up OMA
h5file = open_file(config.omadirLaurent + 'OmaServer.h5', mode="r") 
#setup db objects
dbObj = db.Database(h5file)
omaIdObj = db.OmaIdMapper(dbObj)

# corrects species tree and replacement dictionary for orthoXML files
species_tree, replacement_dic = formatOrtho.fix_species_tree("speciestree.nwk", omaIdObj)

#temporary, 
hashmat_list = [] 
mat_list = []

#load Fam
for fam in pyhamPipeline.yieldFamilies(h5file):

    # generate pyham object
#     print(fam)
    ortho = dbObj.get_orthoxml(fam).decode()
#     print(ortho)
    ortho = formatOrtho.convert_orthoxml_ids(ortho, replacement_dic, verbose = True)
    print(ortho)
    ham_fam = pyhamPipeline.get_ham(fam, dbObj, species_tree, replacement_dic)

    # generate tree profile
    treemap_fam = pyhamPipeline.pyhamtoTree(ham_fam, fam)

    # generate matrix of hash
    hashmat = profileGen.Tree2Hashes(treemap_fam)
    hashmat_list.append(hashmat)
    # generate taxa index
    taxaIndex, taxaIndexReverse = profileGen.generateTaxaIndex(species_tree)

    # generate matrix of 1 and 0 for each biological event
    mat = profileGen.Tree2mat(treemap_fam, taxaIndex)
    mat_list.append(mat)
    print(fam)

    
    if fam > 150:
        break

/home/laurent/anaconda3/lib/python3.6/site-packages/tables/leaf.py:396: PerformanceWarning: The Leaf ``/Protein/_i_Entries/OmaHOG/sorted`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)


  <species name="Haloredivivus sp. (strain G17)" NCBITaxId="1072681">
['1873']
<?xml version='1.0' encoding='UTF-8'?>
<orthoXML xmlns="http://orthoXML.org/2011/" origin="OMA" originVersion="Nov 2017" version="0.3">
  <species name="Nanosalinarum_sp___strain_J07AB56_" NCBITaxId="889962">
    <database name="Candidatus Nanosalinarum sp. J07AB56 (hypersaline lake metagenome)" version="GL982569.1  GI:339756006">
      <genes>
        <gene id="6851" protId="NANSJ01348"/>
      </genes>
    </database>
  </species>
  <species name="Nanosalina_sp___strain_J07AB43_" NCBITaxId="889948">
    <database name="Candidatus Nanosalina sp. J07AB43 (hypersaline lake metagenome)" version="GL982576.1  GI:339757416">
      <genes>
        <gene id="5465" protId="NANS001635"/>
      </genes>
    </database>
  </species>
  <groups>
    <orthologGroup id="1">
      <property name="TaxRange" value="Nanohaloarchaea"/>
      <geneRef id="5465"/>
      <geneRef id="6851"/>
    </orthologGroup>
  </groups>
</orth

/home/laurent/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


ValueError: Minimum 2 genomes are required, only 1 provided.